In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 281.4 MB 42 kB/s 
     |████████████████████████████████| 199 kB 52.9 MB/s 
  Created wheel for pyspark: filename=pyspark-3.3.1-py2.py3-none-any.whl size=281845514 sha256=111518583671756f00e0004cd748dcd6785014106f335d63d5b034c83c8b6835
  Stored in directory: /root/.cache/pip/wheels/42/59/f5/79a5bf931714dcd201b26025347785f087370a10a3329a899c
Successfully built pyspark


In [ ]:
# import SparkContext and SparkSession and other utilities
from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.sql.functions import col, udf, spark_partition_id, explode, count
from pyspark.sql.types import ArrayType, StringType
from pyspark import StorageLevel

In [ ]:
# create SparkSession
spark = SparkSession.builder.master("local").appName('loading').getOrCreate()


In [ ]:
spark

In [ ]:
!ls drive/MyDrive/data/

mapreduce	     World2022-04-27.csv  World2022-04-29.csv
World2022-04-26.csv  World2022-04-28.csv  World2022-04-30.csv


In [ ]:
def eval_list(x):
    return eval(x)

In [ ]:
# read data
df = spark.read.csv(f'drive/MyDrive/data/World2022-04-28.csv', header=False, inferSchema=True)
# make it a pyspark dataframe
df = df.toDF('viewers', 'streamer', 'date', 'category', 'viewership', 'language')
# define the udf
eval_list_udf = udf(eval_list, ArrayType(StringType()))
# apply the udf to the column
df = df.withColumn('viewers', eval_list_udf(col('viewers')))
# drop useless columns
df = df.drop('viewership', 'language', 'date', 'category')
# explode the viewers
df = df.withColumn('viewers', explode(df.viewers))
# group by (viewer, streamer)
df = df \
    .groupBy("viewers", "streamer") \
    .agg(count("viewers").alias("comptage")) \
    .orderBy(col("comptage").desc()) \
    .where(col("comptage") >= 4)
# write csv
df.write.csv("mapreduce", mode='append')

In [ ]:
# read data
df = spark.read.csv(f'drive/MyDrive/data/World2022-04-29.csv', header=False, inferSchema=True)
# make it a pyspark dataframe
df = df.toDF('viewers', 'streamer', 'date', 'category', 'viewership', 'language')
# define the udf
eval_list_udf = udf(eval_list, ArrayType(StringType()))
# apply the udf to the column
df = df.withColumn('viewers', eval_list_udf(col('viewers')))
# drop useless columns
df = df.drop('viewership', 'language', 'date', 'category')
# explode the viewers
df = df.withColumn('viewers', explode(df.viewers))
# group by (viewer, streamer)
df = df \
    .groupBy("viewers", "streamer") \
    .agg(count("viewers").alias("comptage")) \
    .orderBy(col("comptage").desc()) \
    .where(col("comptage") >= 4)
# write csv
df.write.csv("mapreduce", mode='append')

In [ ]:
# read data
df = spark.read.csv(f'drive/MyDrive/data/World2022-04-30.csv', header=False, inferSchema=True)
# make it a pyspark dataframe
df = df.toDF('viewers', 'streamer', 'date', 'category', 'viewership', 'language')
# define the udf
eval_list_udf = udf(eval_list, ArrayType(StringType()))
# apply the udf to the column
df = df.withColumn('viewers', eval_list_udf(col('viewers')))
# drop useless columns
df = df.drop('viewership', 'language', 'date', 'category')
# explode the viewers
df = df.withColumn('viewers', explode(df.viewers))
# group by (viewer, streamer)
df = df \
    .groupBy("viewers", "streamer") \
    .agg(count("viewers").alias("comptage")) \
    .orderBy(col("comptage").desc()) \
    .where(col("comptage") >= 4)
# write csv
df.write.csv("mapreduce", mode='append')

In [ ]:
!ls mapreduce/

_SUCCESS


In [ ]:
!mv mapreduce/*.csv drive/MyDrive/data/mapreduce/

# STEP BY STEP

In [ ]:
df = spark.read.csv('drive/MyDrive/data/World2022-04-1.csv', header=False, inferSchema=True)

In [ ]:
df = df.toDF('viewers', 'streamer', 'date', 'category', 'viewership', 'language')


In [ ]:
df.show()

+--------------------+---------------+-------------------+-------------------+----------+--------+
|             viewers|       streamer|               date|           category|viewership|language|
+--------------------+---------------+-------------------+-------------------+----------+--------+
|['0000moonlight',...|          Emiru|2022-04-01 00:00:03|      Just Chatting|     45368|      en|
|['000_fnwortas_00...|       ROSHTEIN|2022-04-01 00:00:03|              Slots|     34077|      en|
|['000jp', '00od',...|       HasanAbi|2022-04-01 00:00:03|      Just Chatting|     30861|      en|
|['000008', '003ra...|TSM_ImperialHal|2022-04-01 00:00:03|       Apex Legends|     29138|      en|
|['0000_kevin0000'...|       alanzoka|2022-04-01 00:00:03|Midnight Ghost Hunt|     28314|      pt|
|['007dosul_salve'...|         Gaules|2022-04-01 00:00:03|             Sports|     27691|      pt|
|['00xhe', '03gpwa...|      loltyler1|2022-04-01 00:00:03|  League of Legends|     24670|      en|
|['000_jra

In [ ]:
# print sum of all viewership
# print(df.select('viewership').agg({'viewership': 'sum'}).collect()[0][0])

In [ ]:
# define the function to be applied to each row
def eval_list(x):
    return eval(x)

In [ ]:
# define the udf
eval_list_udf = udf(eval_list, ArrayType(StringType()))
# apply the udf to the column
df = df.withColumn('viewers', eval_list_udf(col('viewers')))


In [ ]:
df.printSchema()

root
 |-- viewers: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- streamer: string (nullable = true)
 |-- date: timestamp (nullable = true)
 |-- category: string (nullable = true)
 |-- viewership: integer (nullable = true)
 |-- language: string (nullable = true)



In [ ]:
df = df.drop('viewership', 'language', 'date', 'category')

In [ ]:
df = df.withColumn('viewers', explode(df.viewers))

In [ ]:
df = df \
    .groupBy("viewers", "streamer") \
    .agg(count("viewers").alias("comptage")) \
    .orderBy(col("comptage").desc()) \
    .where(col("comptage") >= 4)

In [ ]:
# df.show()

In [ ]:
# print((df.count(), len(df.columns)))

In [ ]:
# write to csv
df.write.csv("mapreduce", mode='overwrite')

In [ ]:
!ls mapreduce/

_SUCCESS


In [ ]:
!ls drive/MyDrive/data/mapreduce/

part-00000-0245cfd2-b7c4-4434-8e37-b46b37834da0-c000.csv
part-00000-3e08f1b1-45f5-445a-86cf-65b4783ef2ff-c000.csv
part-00000-612e4333-da51-4750-94ad-eb485dc6a2b3-c000.csv
part-00000-844ca738-61fd-4b56-b1a1-2a9ea9af19a3-c000.csv
part-00000-fe8edf08-4238-4f0b-a33a-3212248e1048-c000.csv
part-00001-0245cfd2-b7c4-4434-8e37-b46b37834da0-c000.csv
part-00001-3e08f1b1-45f5-445a-86cf-65b4783ef2ff-c000.csv
part-00001-612e4333-da51-4750-94ad-eb485dc6a2b3-c000.csv
part-00001-844ca738-61fd-4b56-b1a1-2a9ea9af19a3-c000.csv
part-00001-fe8edf08-4238-4f0b-a33a-3212248e1048-c000.csv
part-00002-0245cfd2-b7c4-4434-8e37-b46b37834da0-c000.csv
part-00002-3e08f1b1-45f5-445a-86cf-65b4783ef2ff-c000.csv
part-00002-612e4333-da51-4750-94ad-eb485dc6a2b3-c000.csv
part-00002-844ca738-61fd-4b56-b1a1-2a9ea9af19a3-c000.csv
part-00002-fe8edf08-4238-4f0b-a33a-3212248e1048-c000.csv
part-00003-0245cfd2-b7c4-4434-8e37-b46b37834da0-c000.csv
part-00003-844ca738-61fd-4b56-b1a1-2a9ea9af19a3-c000.csv


In [ ]:
!mv mapreduce/*.csv drive/MyDrive/data/mapreduce/